In [26]:
import pandas as pd
import numpy as np
from google.colab import drive

# ----------------------------------------------------
# 1. SETUP DAN MEMUAT DATA
# ----------------------------------------------------
try:
    drive.mount('/content/drive')
except:
    print("Drive sudah ter-mount atau gagal mount. Melanjutkan...")

file_path = '/content/drive/MyDrive/kuliah ITN/Semester 5/Data Mining/car_sales_dirty.csv'
df = pd.read_csv(file_path, sep=';')

print("\n--- Inspeksi Awal Data ---")
print("Data Kosong (NaN) Awal di semua kolom:")
print(df.isnull().sum())
print("-" * 50)


# ----------------------------------------------------
# 2. PRA-CLEANSING DAN KONVERSI AWAL (Mempersiapkan Kalkulasi Median/Mode)
# ----------------------------------------------------

# --- Konversi Numerik (Engine size, Price, Year, Mileage) ---
# Engine Size: Ganti koma dengan titik, lalu konversi ke numerik
df['Engine size'] = df['Engine size'].astype(str).str.replace(',', '.', regex=False)
df['Engine size'] = pd.to_numeric(df['Engine size'], errors='coerce')

# Konversi dan Coerce lainnya
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df['Year of manufacture'] = pd.to_numeric(df['Year of manufacture'], errors='coerce')
df['Mileage'] = pd.to_numeric(df['Mileage'], errors='coerce')


# --- CLEANSING DAN KALKULASI NILAI IMPUTASI (Mode/Median) ---

# Kategorikal (Mode Imputation)
df['Manufacturer'] = df['Manufacturer'].astype(str).str.strip().str.title()
df['Manufacturer'].replace('Nan', np.nan, inplace=True)
mode_manufacturer = df['Manufacturer'].mode()[0]

df['Model'] = df['Model'].astype(str).str.strip().str.replace('@', '', regex=False).str.title()
df['Model'].replace('Nan', np.nan, inplace=True)
df['Model'].replace('', np.nan, inplace=True)
mode_model = df['Model'].mode()[0]

df['Fuel type'] = df['Fuel type'].astype(str).str.strip().str.lower()
df['Fuel type'].replace('nan', np.nan, inplace=True)
mode_fuel_type = df['Fuel type'].mode()[0]

# Numerik (Median Imputation)
median_engine = df['Engine size'].median()
median_price = df['Price'].median()
median_year = df['Year of manufacture'].median()
median_mileage = df['Mileage'].median()

print("--- Nilai Imputasi yang Digunakan (Mode/Median) ---")
print(f"Manufacturer (Mode): {mode_manufacturer}")
print(f"Price (Median): {median_price}")
print(f"Fuel type (Mode): {mode_fuel_type}")
print("-" * 30)


# ----------------------------------------------------
# 3. IMPUTASI FINAL (Mengisi NaN dengan Nilai Bersih)
# ----------------------------------------------------

# Imputasi Kategorikal
df['Manufacturer'].fillna(mode_manufacturer, inplace=True)
df['Model'].fillna(mode_model, inplace=True)
df['Fuel type'].fillna(mode_fuel_type, inplace=True)

# Imputasi Numerik
df['Engine size'].fillna(median_engine, inplace=True)
df['Price'].fillna(median_price, inplace=True)
df['Year of manufacture'].fillna(median_year, inplace=True)
df['Mileage'].fillna(median_mileage, inplace=True)

# Post-Imputasi: Konversi kolom integer (agar tidak ada angka desimal pada kolom bulat)
df['Year of manufacture'] = df['Year of manufacture'].astype(int)
df['Mileage'] = df['Mileage'].astype(int)
# Konversi Price ke integer (angka bulat)
df['Price'] = df['Price'].astype(int)


# ==========================================================
# 4. STANDARDISASI & FORMATTING PRICE KE BENTUK USD ($)
# ==========================================================

print("\n--- Standardisasi Price ke Format USD ---")

# Membuat kolom baru untuk format visual USD ($)
# Catatan: Kolom 'Price' yang asli tetap numerik untuk analisis.
# Kolom baru ini hanya untuk tampilan.
df['Price (USD)'] = df['Price'].apply(lambda x: f"${x:,.0f}")
# Kolom Price (USD) akan terlihat seperti: $68,597

print("✅ Kolom 'Price (USD)' berhasil ditambahkan.")

# ----------------------------------------------------
# 5. HASIL AKHIR DAN PENYIMPANAN
# ----------------------------------------------------

print("\n--- Verifikasi Hasil Akhir Cleansing & Standardisasi ---")

# Cek jumlah NaN pada SEMUA kolom
print("✅ Jumlah Data Kosong (NaN) di SEMUA kolom:")
print(df.isnull().sum())

print("-" * 30)
print("5 Data Teratas Setelah Pembersihan & Format USD:")
print(df[['Manufacturer', 'Model', 'Price', 'Price (USD)']].head())


# ===============================================
# 6. CODE UNTUK MENYIMPAN FILE BERSIH
# ===============================================

output_file_path = '/content/drive/MyDrive/kuliah ITN/Semester 5/Data Mining/car_sales_CLEANSED_IMPUTED_FINAL_USD.csv'

try:
    df.to_csv(output_file_path, index=False, sep=';')
    print(f"\n✅ File Berhasil Disimpan!")
    print(f"File bersih (dengan format USD) tersedia di: {output_file_path}")

except Exception as e:
    print(f"\n❌ Terjadi kesalahan saat menyimpan file: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

--- Inspeksi Awal Data ---
Data Kosong (NaN) Awal di semua kolom:
Manufacturer           4
Model                  6
Engine size            6
Fuel type              6
Year of manufacture    6
Mileage                6
Price                  6
dtype: int64
--------------------------------------------------
--- Nilai Imputasi yang Digunakan (Mode/Median) ---
Manufacturer (Mode): Ford
Price (Median): 6765.0
Fuel type (Mode): petrol
------------------------------

--- Standardisasi Price ke Format USD ---
✅ Kolom 'Price (USD)' berhasil ditambahkan.

--- Verifikasi Hasil Akhir Cleansing & Standardisasi ---
✅ Jumlah Data Kosong (NaN) di SEMUA kolom:
Manufacturer           0
Model                  0
Engine size            0
Fuel type              0
Year of manufacture    0
Mileage                0
Price                  0
Price (USD)            0
dtype: int64
-------

/tmp/ipython-input-1697073714.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Manufacturer'].replace('Nan', np.nan, inplace=True)
/tmp/ipython-input-1697073714.py:45: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)